In [11]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, GlueDataset, default_data_collator
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import mlflow
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../Data/train_df.csv")
data = data.dropna(subset=['words'])
data = data.sample(1000)

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 128

/Applications/anaconda3/envs/BERT-test/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def encode_text(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_len,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt',
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

In [5]:
input_ids, attention_masks = encode_text(data['words'], tokenizer, max_len)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, data['target'].values, test_size=0.1, random_state=42)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.1, random_state=42)

In [6]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
eval_dataset = TensorDataset(validation_inputs, validation_masks, validation_labels)

/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_11724/2933568192.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_11724/2933568192.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_inputs = torch.tensor(validation_inputs)
/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_11724/2933568192.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_masks = torch.tensor(train_masks)
/var/folders/kl/3nc79ycx61

In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def my_data_collator(features):
    batch = {}
    batch['input_ids'] = torch.stack([f[0] for f in features])
    batch['attention_mask'] = torch.stack([f[1] for f in features])
    batch['labels'] = torch.tensor([f[2] for f in features])
    return batch

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    warmup_steps=0,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=my_data_collator
)

In [ ]:
trainer.train()

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Supposons que 'model' est votre modèle BERT entraîné
# Supposons que 'tokenizer' est votre tokenizer associé

# Chemin pour sauvegarder le modèle et le tokenizer
model_path = "./model"
tokenizer_path = "./tokenizer"

# Sauvegarder le modèle
model.save_pretrained(model_path)

# Sauvegarder le tokenizer
tokenizer.save_pretrained(tokenizer_path)

In [12]:
tokenizer_path = "./tokenizer"
model_path = "./model"

tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [13]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
text = "Hello, how are you?"

result = classifier(text)
print(result)

[{'label': 'LABEL_0', 'score': 0.7381207346916199}]


In [19]:
text = "I want you to die"

result = classifier(text)
print(result)

[{'label': 'LABEL_0', 'score': 0.7199581265449524}]


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Charger les données et le modèle entraîné
data = pd.read_csv("../Data/train_df.csv")
data = data.dropna(subset=['words'])
data = data.sample(1000)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("chemin/vers/votre/modele")

# Prétraiter les données de test
test_texts = ["Hello, how are you?"]  # Mettez vos propres données de test ici
test_input_ids = []
test_attention_masks = []
for text in test_texts:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        max_length=128,
                        padding='max_length',
                        return_attention_mask=True,
                        return_tensors='pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

# Effectuer les prédictions
with torch.no_grad():
    outputs = model(test_input_ids, attention_mask=test_attention_masks)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Calculer les métriques de performance
accuracy = accuracy_score(data['target'], predictions)
precision = precision_score(data['target'], predictions)
recall = recall_score(data['target'], predictions)
f1 = f1_score(data['target'], predictions)
fpr, tpr, thresholds = roc_curve(data['target'], predictions)
roc_auc = auc(fpr, tpr)
conf_matrix = confusion_matrix(data['target'], predictions)

# Enregistrer les résultats dans MLflow
import mlflow

artifact_path = './artifacts/'

with mlflow.start_run(run_name="bert-base-uncased"):
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1_Score", f1)
    mlflow.log_metric("AUC", roc_auc)

    conf_matrix_path = f"{artifact_path}confusion_matrix.csv"
    pd.DataFrame(conf_matrix).to_csv(conf_matrix_path, index=False, header=False)
    mlflow.log_artifact(conf_matrix_path, "metrics")

    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True)
    roc_curve_path = f"{artifact_path}roc_curve.png"
    plt.savefig(roc_curve_path)
    plt.close()
    mlflow.log_artifact(roc_curve_path, "plots")
